# Лабораторная работа 1

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков Jupyter (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий, а также подкреплённый грамотной визуализацией. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам —  проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета, кода и графиков. 

## Оценивание и штрафы

* Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи)
* Максимально допустимая оценка за работу — 15 баллов
* Сдавать задание после указанного срока сдачи нельзя
* «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают свою карму
* Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому, чтобы исключить подозрение в плагиате, необходима ссылка на источник)
* Не оцениваются задания с удалёнными формулировкам
* Не оценивается лабораторная работа целиком, если она была выложена в открытый источник

## Данные

В данной лабораторной работе вы будете работать с данными из Dota2. 

Скачать датасет и подробно ознакомится с его описанием вы можете на странице первого конкурсного [соревнования](https://www.kaggle.com/t/8ddaf99c566b4d088b3e352ce1bbbc19).

## Маштабирование признаков

Масштабирование признаков можно выполнить, например, одним из следующих способов:
 - $x_{new} = \dfrac{x - \mu}{\sigma}$, где $\mu, \sigma$ — среднее и стандартное отклонение значения признака по всей выборке (см. функцию [scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html))
 - $x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, где $[x_{min}, x_{max}]$ — минимальный интервал значений признака

Похожие схемы масштабирования приведены в классах [sklearn.preprocessing.StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) и [sklearn.preprocessing.MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler).

## Подбор гиперпараметров модели

В задачах машинного обучения следует различать параметры модели и гиперпараметры (структурные параметры). Обычно параметры модели настраиваются в ходе обучения (например, веса в линейной модели), в то время как гиперпараметры задаются заранее (например, значение силы регуляризации в линейной модели). Каждая модель, как правило, имеет множество гиперпараметров и нет универсальных наборов гиперпараметров, оптимально работающих во всех задачах, поэтому для каждой задачи нужно подбирать свой набор.

Для оптимизации гиперпараметров модели часто используют _перебор по сетке (grid search)_: для каждого гиперпараметра выбирается несколько значений, далее перебираются все комбинации значений и выбирается комбинация, на которой модель показывает лучшее качество (с точки зрения оптимизируемой метрики). Однако, в этом случае нужно грамотно оценивать построенную модель, а именно делать разбиение на обучающую и тестовую выборку.

В этом случае сравнение большого числа моделей при переборе гиперпараметров приводит к ситуации, когда лучшая на тестовой подвыборке модель не сохраняет свои качества на новых данных. Можно сказать, что происходит _переобучение_ на тестовую выборку.

Для устранения описанной выше проблемы, **можно разбить данные на 3 непересекающихся подвыборки: обучение, валидация и тест**. Валидационную подвыборку используют для сравнения моделей, а тестовую — для окончательной оценки качества и сравнения семейств моделей с подобранными гиперпараметрами.

**Другой способ сравнения моделей — [кросс-валидация](http://bit.ly/1CHXsNH)**. 

Существуют различные **схемы кросс-валидации**:
  - Leave-One-Out
  - K-Fold
  - Многократное случайное разбиение выборки
  
Кросс-валидация вычислительно затратна, особенно если вы делаете перебор по сетке с очень большим числом комбинации. С учетом конечности времени на выполнение задания, возникает ряд компромиссов: 
  - сетку гиперпараметров можно делать более разреженной, перебирая меньше значений каждого гиперпараметра; однако, не стоит забывать, что в таком случае можно пропустить хорошую комбинацию гиперпараметров;
  - кросс-валидацию можно делать с меньшим числом разбиений или фолдов, но в таком случае оценка качества становится более шумной и увеличивается риск выбрать неоптимальный набор гиперпараметров из-за случайности разбиения;
  - гиперпараметры можно оптимизировать последовательно (жадно) — один за другим, а не перебирать все комбинации; такая стратегия не всегда приводит к оптимальному набору;
  - перебирать не все комбинации гиперпараметров, а небольшое число случайно выбранных.

## Линейная регрессия и SGD

В данном блоке вам предстоит предсказать суммарное количество золота для команды `radiant` 

(игроки: `player_0`, `player_1`, `player_2`, `player_3`, `player_4`) на 600 секунде матча.

In [1]:
import pandas as pd

X = pd.read_csv('Dataset/public/gold.csv', index_col=[0, 1])

radiant_team = ['player_0', 'player_1', 'player_2', 'player_3', 'player_4']
dire_team = ['player_5', 'player_6', 'player_7', 'player_8', 'player_9']

Y = X.loc[pd.IndexSlice[:, 600], radiant_team].sum(axis=1).droplevel('times')
X = X.unstack().drop([(player, 600) for player in radiant_team], axis=1)

X.columns = ['{}_{}'.format(*column) for column in X.columns]

Для оценки качества предсказания будем использовать [sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html). 

**Задание 1** (3 балла)

Реализуйте класс `CustomSGDRegressor`, который бы обучал многомерную линейную регрессию минимизируя сумму квадратов ошибки, используя `SGD`.

Класс, должен принимать следующие параметры при инициализации:
    
    learning_rate : float : параметр задающий скорось обучения
    fit_intercept : bool : если True, то добавляем свободный член
    max_iter : int : максимальное число эпох
    shuffle : bool : если True, то перемещиваем данные обучения перед каждой эпохой
    
и иметь методы `fit` и `predict`.

После обучения (запуска метода `fit`) мы должны мочь обратиться к атрибутам класса: 
    
    coef_  : ndarray of shape (n_features,)
    intercept_ : ndarray of shape (1,)

In [ ]:
class CustomSGDRegressor
    ...

Разбейте `X` и `Y` на `train` и `test`, используя [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html).

Далее вам нужно подобрать такой `learning_rate`, на котором достигается минимум среднеквадратичная ошибка, 

а также принять решение о необходимости масштабирования признаков.

Постройте соответствующие кривые обучения и сделайте выводы.

Обучите [sklearn.linear_model.SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html) выставив его параметры при инициализации так, что бы он максимально был близок к реализованной вами модели.

Сравните вашу реализацию и реализацию из sklearn: 
    
    1. Достигают ли они сравнимого качества на данном наборе данных?
    2. Одинаковая ли у них скорость сходимости? 
    3. Одинаковая ли скорость обучения при заданном количестве эпох?
    4. Похожие ли веса модели были найдены?
    
Если ваша реализация сильно отличается по данным параметрам от реализации из sklearn, то проанализируйте почему.

**Задание 2** (2 балла)

Повысьте качество модели `SGDRegressor` проведя эксперименты со стратегией обучения, а также с регуляризацией. 

Для подбора гиперпараметров воспользуйтесь перебором по сетке, который реализован в классе [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). В качестве схемы кросс-валидации используйте 5-Fold CV, которую можно задать с помощью класса [KFoldCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

Постройте график среднего значения качества по кросс-валидации алгоритма при заданном значении гиперпараметра, на котором также отобразите доверительный интервал. Для получения значения качества на каждом фолде, среднего значение качества и другой полезной информации можно воспользоваться полем [*cv results_*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

Какая комбинация стратегии обучения и регуляризации оказалась выигрышной?

**Задание 3** (3 балла)

Попробуйте обогатить `X` данными из остальных таблиц, так что бы повысить качество модели `SGDRegressor`.

Проведите ряд экспериментов, целью которых будет выявить оптимальный набор данных для решения данной задачи. 

Все эксперименты должны быть подкреплены корректными и понятными графиками.

Опишите вашу стратегию добавления признаков и оценки их значимости, а также ответьте на вопросы:

1. Удалось ли повысить качество предсказания за счёт добавления новых признаков?
2. Есть ли признаки, добавление которых, ухудшило качество модели? Как вы думаете, почему так произошло?

## Бинарная классификация

**Задание 4** (3 балла)

Реализуйте класс `CustomSGDClassifier`, который бы обучал логистическую регрессию, используя `SGD`.

Класс, должен принимать следующие параметры при инициализации:
    
    learning_rate : float : параметр задающий скорось обучения
    fit_intercept : bool : если True, то добавляем свободный член
    max_iter : int : максимальное число эпох
    shuffle : bool : если True, то перемещиваем данные обучения перед каждой эпохой
    
и иметь методы `fit`, `predict` и `predict_proba`.

После обучения (запуска метода `fit`) мы должны мочь обратиться к атрибутам класса: 
    
    coef_  : ndarray of shape (n_features,)
    intercept_ : ndarray of shape (1,)

In [ ]:
class CustomSGDClassifier
    ...

Используя `CustomSGDClassifier` научитесь прогнозировать победу команды Radiant.

Оптимальный набор гиперпараметров и признаков отберите, опираясь на такие метрики, как `precision`,  `recall` и `ROC-AUC`.
    
Все эксперименты должны быть подкреплены корректными и понятными графиками.

**Задание 5** (2 балла)

Попробуйте повысить качество предсказания, используя:

 - [Logistic Regression Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
 - [C-Support Vector Classification](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
 - [Multi-layer Perceptron Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)
 
Проведите эксперементы по подбору гиперпараметров. Какой алгоритм показал наилучшее качество?

Может ли масштабирование повлиять на работу этих алгоритмов?

### Сравнение построенных моделей

![](http://cdn.shopify.com/s/files/1/0870/1066/files/compare_e8b89647-3cb6-4871-a976-2e36e5987773.png?1750043340268621065)

После того как было построено много моделей, правильным продолжением является сравнение их между собой.  Воспользуйтесь диаграммой размаха ("ящик с усами") для сравнения алгоритмов между собой. Эту диаграмму можно построить при помощи [boxplot](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.boxplot.html) из matplotlib либо через [обертку](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html) над ней из pandas.

**Задание 6** (2 балла)

Для каждого типа классификатора, выберите тот, которых давал наилучшее качество на кросс-валидации и постройте диаграмму размаха. Все классификаторы должны быть изображены на одном графике.
 
Сделайте общие итоговые выводы о классификаторах с точки зрения их работы с признаками и сложности самой модели (какие гиперпараметры есть у модели, сильно ли изменение значения гиперпараметра влияет на качество модели).